In [1]:
import models
import utils
import data_processing
from constants import (
    VALUE_COLUMN,
    UNIT_RATE_COLUMN,
    QUANTITY_COLUMN,
    GROSS_WEIGHT_COLUMN,
    SHIP_COUNT_COLUMN,
    PORT_COUNT_COLUMN,
    FILL_METHOD,
    TARGET_COLUMN,
    VOLZA_COLUMNS,
    AIS_COLUMNS,
    OIL_COLUMNS,
    ARIMA_RESIDUAL_COLUMN,
    RANDOM_STATE
)
import pandas as pd
import constants
import sys
from datetime import datetime
from build_data import get_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

import matplotlib.pyplot as plt
import numpy as np

In [2]:
COMMODITYS = ['cobalt', 'copper', 'germanium', 'magnesium']
#target_COMMODITY = "copper"
target_COMMODITY = "copper"
WINDOW_SIZE = 20

In [3]:
pre_features = []
pre_labels = []
tar_features = []
tar_labels = []
for COMMODITY in COMMODITYS:
    VOLZA_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}.csv"
    PRICE_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}_prices.csv"
    
    # Get the data
    data = get_data(VOLZA_FILE_PATH, PRICE_FILE_PATH, window_size=20, center=False)
    
    # Add Isolation Forest spikes column
    data['spikes_if'] = utils.detect_spikes_if(data, TARGET_COLUMN, contamination=0.1)
    
    # Add Bowen's spike detection
    _, features, labels = utils.detect_spikes_new(data, TARGET_COLUMN, window_size=WINDOW_SIZE, center=False)
    if COMMODITY == target_COMMODITY:
        tar_features.extend(features)
        tar_labels.extend(labels)
        continue
    pre_features.extend(features)
    pre_labels.extend(labels)

Building data...


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\analysis\build_data.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={"Value": "Brent Oil Value"}, inplace=True)
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\analysis\build_data.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={"Value": "WTI Oil Value"}, inplace=True)
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\analysis\build_data.py:69: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() inste

Rows dropped due to NaN values: 1


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Detecting spikes... 20
Building data...


...

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Detecting spikes... 20
Building data...


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\analysis\build_data.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={"Value": "Brent Oil Value"}, inplace=True)
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\analysis\build_data.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={"Value": "WTI Oil Value"}, inplace=True)
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\analysis\build_data.py:69: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() inste

Rows dropped due to NaN values: 1
Detecting spikes... 20


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [4]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomOverSampler
output_file_path = f'{target_COMMODITY}/test/results_test.csv'
pred_file_path = f'{target_COMMODITY}/test/predictions/test'
model_path = f'{target_COMMODITY}/best_model'

In [5]:
aggregated_df = data
SPIKE_COLUMN = 'spikes_if'

In [6]:
# Prepare price data
# X_price, y_price = data_processing.prepare_features_and_target(aggregated_df, TARGET_COLUMN, SPIKE_COLUMN)
X_price = np.array(tar_features)
y_price = np.array(tar_labels)

# Split price data
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(X_price, y_price, test_size=0.3, random_state=RANDOM_STATE, shuffle=False)
X_train_price, X_val_price, y_train_price, y_val_price = train_test_split(X_train_price, y_train_price, test_size=(10/70), random_state=RANDOM_STATE, shuffle=False)

# Balancing
X_train_price, y_train_price = sampler(random_state=RANDOM_STATE).fit_resample(X_train_price, y_train_price)

# Scaling
X_train_price, X_test_price, X_val_price = data_processing.scale_features(X_train_price, X_test_price, X_val_price)

# Sequence making
# X_train_price, y_train_price = data_processing.create_sequences(X_train_price, y_train_price, WINDOW_SIZE)
# X_test_price, y_test_price = data_processing.create_sequences(X_test_price, y_test_price, WINDOW_SIZE)
# X_val_price, y_val_price = data_processing.create_sequences(X_val_price, y_val_price, WINDOW_SIZE)


X_train_price = np.expand_dims(X_train_price, axis = 2)
X_test_price = np.expand_dims(X_test_price, axis = 2)
X_val_price = np.expand_dims(X_val_price, axis = 2)

In [7]:
results_df  = models.evaluate_all(X_train_price, y_train_price, X_val_price, y_val_price, X_test_price, y_test_price, output_file_path, pred_file_path, model_path, False, False)



7/7 [==============================] - 0s 9ms/step
Predictions saved to CSV file: copper/test/predictions/test/LSTM_256_layers_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test/LSTM_256_layers_predictions.npy
Predictions saved to CSV file: copper/test/predictions/test_confidence/LSTM_256_layers_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test_confidence/LSTM_256_layers_predictions.npy
{'Name': 'LSTM', 'Params': '256 layers', 'Accuracy': 0.6081081081081081, 'Precision (0)': 0.8424657534246576, 'Recall (0)': 0.6577540106951871, 'F1 (0)': 0.7387387387387387, 'Precision (1)': 0.15789473684210525, 'Recall (1)': 0.34285714285714286, 'F1 (1)': 0.21621621621621623, 'Prior': '0.16'}


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 0s 5ms/step
Predictions saved to CSV file: copper/test/predictions/test/LSTM_128_layers_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test/LSTM_128_layers_predictions.npy
Predictions saved to CSV file: copper/test/predictions/test_confidence/LSTM_128_layers_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test_confidence/LSTM_128_layers_predictions.npy
{'Name': 'LSTM', 'Params': '128 layers', 'Accuracy': 0.7882882882882883, 'Precision (0)': 0.8608247422680413, 'Recall (0)': 0.893048128342246, 'F1 (0)': 0.8766404199475065, 'Precision (1)': 0.2857142857142857, 'Recall (1)': 0.22857142857142856, 'F1 (1)': 0.25396825396825395, 'Prior': '0.16'}


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 0s 1ms/step
Predictions saved to CSV file: copper/test/predictions/test/LSTM_64_layers_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test/LSTM_64_layers_predictions.npy
Predictions saved to CSV file: copper/test/predictions/test_confidence/LSTM_64_layers_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test_confidence/LSTM_64_layers_predictions.npy
{'Name': 'LSTM', 'Params': '64 layers', 'Accuracy': 0.6531531531531531, 'Precision (0)': 0.8481012658227848, 'Recall (0)': 0.7165775401069518, 'F1 (0)': 0.7768115942028986, 'Precision (1)': 0.171875, 'Recall (1)': 0.3142857142857143, 'F1 (1)': 0.22222222222222224, 'Prior': '0.16'}


...

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 0s 2ms/step
Predictions saved to CSV file: copper/test/predictions/test/CNN_256_filters_5_kernels_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test/CNN_256_filters_5_kernels_predictions.npy
Predictions saved to CSV file: copper/test/predictions/test_confidence/CNN_256_filters_5_kernels_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test_confidence/CNN_256_filters_5_kernels_predictions.npy
CNN_256_filters_5_kernels {'Name': 'CNN', 'Params': '256 filters, kernel size 5', 'Accuracy': 0.7252252252252253, 'Precision (0)': 0.8539325842696629, 'Recall (0)': 0.8128342245989305, 'F1 (0)': 0.8328767123287671, 'Precision (1)': 0.20454545454545456, 'Recall (1)': 0.2571428571428571, 'F1 (1)': 0.22784810126582275, 'Prior': '0.16'}


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 0s 1ms/step
Predictions saved to CSV file: copper/test/predictions/test/CNN_256_filters_3_kernels_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test/CNN_256_filters_3_kernels_predictions.npy
Predictions saved to CSV file: copper/test/predictions/test_confidence/CNN_256_filters_3_kernels_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test_confidence/CNN_256_filters_3_kernels_predictions.npy
CNN_256_filters_3_kernels {'Name': 'CNN', 'Params': '256 filters, kernel size 3', 'Accuracy': 0.8153153153153153, 'Precision (0)': 0.8476190476190476, 'Recall (0)': 0.9518716577540107, 'F1 (0)': 0.8967254408060453, 'Precision (1)': 0.25, 'Recall (1)': 0.08571428571428572, 'F1 (1)': 0.1276595744680851, 'Prior': '0.16'}
Predictions saved to CSV file: copper/test/predictions/test/Dumb_Model_spikes_predictions.csv
Predictions saved to NPY file: copper/test/predictions/test/Dumb_Model_spikes_predictions.npy
Predictions

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

In [8]:
results_df

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,LSTM,256 layers,0.608108,0.842466,0.657754,0.738739,0.157895,0.342857,0.216216,0.16
1,LSTM,128 layers,0.788288,0.860825,0.893048,0.876640,0.285714,0.228571,0.253968,0.16
2,LSTM,64 layers,0.653153,0.848101,0.716578,0.776812,0.171875,0.314286,0.222222,0.16
3,LSTM,32 layers,0.252252,0.769231,0.160428,0.265487,0.142077,0.742857,0.238532,0.16
4,CNN with Attention,"32 filters, kernel size 7",0.594595,0.825503,0.657754,0.732143,0.123288,0.257143,0.166667,0.16
5,CNN with Attention,"32 filters, kernel size 5",0.531532,0.816794,0.572193,0.672956,0.120879,0.314286,0.174603,0.16
6,CNN with Attention,"32 filters, kernel size 3",0.779279,0.838235,0.914439,0.874680,0.111111,0.057143,0.075472,0.16
7,CNN with Attention,"64 filters, kernel size 7",0.707207,0.846591,0.796791,0.820937,0.173913,0.228571,0.197531,0.16
8,CNN with Attention,"64 filters, kernel size 5",0.738739,0.848649,0.839572,0.844086,0.189189,0.200000,0.194444,0.16
9,CNN with Attention,"64 filters, kernel size 3",0.418919,0.802083,0.411765,0.544170,0.126984,0.457143,0.198758,0.16
